# Weekly activity
1. Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use _"lena.jfif"_ as the input image.
    - Use external libraries `imutils`.  
    - Modify the transformation matrix.
2. Use the images with titles: _"flower.jfif"_ and _"native-bee.png"_. I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are `cv.bitwise_and()`, `cv.bitwise_or()` and `cv.bitwise_not()`. You need to use `cv.threshold` function to segment the flower. Please refer to [online documentation](https://docs.opencv.org/4.x/d0/d86/tutorial_py_image_arithmetics.html) for more info. The result should resemble the following:  
![bee and flowers](img_embed/activity3.PNG "bee_flower")
3. Write a function that randomly crop the central region of an image. The method signature should be as shown in the following:
```
random_center_crop(image, min_crop_ratio, max_crop_ratio)
```

4. Aside from Gaussian noise, name another common type of noise. Write the code to demonstrate how the noise can be included in an image.

In [2]:
pip install imutils

Note: you may need to restart the kernel to use updated packages.


In [10]:
#Exercise 1
# Load the image
image_path = 'lena.jfif'
image = cv2.imread(image_path)

# Rotate the image by 45 degrees without cropping sides
rotated = imutils.rotate_bound(image, 45)

# Save and display the rotated image as .jpg
output_path = 'rotated_lena.jpg'
cv2.imwrite(output_path, rotated)
print(f"Image saved as '{output_path}'")

cv2.imshow('Rotated Image', rotated)
cv2.waitKey(0)
cv2.destroyAllWindows()

Image saved as 'rotated_lena.jpg'


In [48]:
#Exercise 2

import cv2
import numpy as np

# Load the images
flower = cv2.imread('images/flower.jfif')
bee = cv2.imread('images/native-bee.png')

# Resize the flower image to a smaller size
flower_height, flower_width = flower.shape[:2]
new_size = (int(flower_width * 1), int(flower_height * 1))  # Scale down by 50%
flower = cv2.resize(flower, new_size)

# Position of the flower (top-left corner)
x, y = 0, 0  # Change these values to move the flower

# Convert flower image to grayscale
flower_gray = cv2.cvtColor(flower, cv2.COLOR_BGR2GRAY)

# Create a binary mask of the flower image
_, mask = cv2.threshold(flower_gray, 60, 255, cv2.THRESH_BINARY)

# Invert the mask
mask_inv = cv2.bitwise_not(mask)

# Define the ROI on the bee image
roi = bee[y:y + flower.shape[0], x:x + flower.shape[1]]

# Black-out the area of the flower in the ROI
bee_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

# Take only region of the flower from the flower image
flower_fg = cv2.bitwise_and(flower, flower, mask=mask)

# Put flower in the ROI and modify the main image
combined_roi = cv2.add(bee_bg, flower_fg)
bee[y:y + flower.shape[0], x:x + flower.shape[1]] = combined_roi

# Save and display the result
cv2.imwrite('bee_and_flower.png', bee)
cv2.imshow('Combined Image', bee)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [52]:
#Exercise 3
import cv2
import numpy as np

def random_center_crop(image, min_crop_ratio, max_crop_ratio):
    """
    Randomly crops the central region of an image.

    Args:
        image (np.ndarray): The input image.
        min_crop_ratio (float): Minimum ratio of crop size to original size.
        max_crop_ratio (float): Maximum ratio of crop size to original size.

    Returns:
        np.ndarray: The cropped image.
    """
    # Get image dimensions
    height, width = image.shape[:2]

    # Randomly determine the crop size
    crop_ratio = np.random.uniform(min_crop_ratio, max_crop_ratio)
    crop_width = int(width * crop_ratio)
    crop_height = int(height * crop_ratio)

    # Calculate the top-left corner of the crop box
    x = np.random.randint(0, width - crop_width)
    y = np.random.randint(0, height - crop_height)

    # Perform the crop
    cropped_image = image[y:y + crop_height, x:x + crop_width]
    return cropped_image


image = cv2.imread('images/native-bee.png')

# Set crop ratios
min_ratio = 0.3  # Minimum crop ratio
max_ratio = 0.8  # Maximum crop ratio

# Crop the image
cropped_image = random_center_crop(image, min_ratio, max_ratio)

# Display the result
cv2.imshow('Bee Image', cropped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [54]:
#Exercise 4 Salt-and-Pepper Noise Function

import cv2
import numpy as np

def add_salt_and_pepper_noise(image, salt_prob, pepper_prob):
    """
    Adds salt-and-pepper noise to an image.

    Args:
        image (np.ndarray): The input image.
        salt_prob (float): Probability of adding salt noise.
        pepper_prob (float): Probability of adding pepper noise.

    Returns:
        np.ndarray: The noisy image.
    """
    noisy_image = np.copy(image)
    total_pixels = image.size

    # Salt noise
    num_salt = int(total_pixels * salt_prob)
    salt_coords = [np.random.randint(0, i - 1, num_salt) for i in image.shape]
    noisy_image[salt_coords[0], salt_coords[1]] = 1  # Salt noise is white (1)

    # Pepper noise
    num_pepper = int(total_pixels * pepper_prob)
    pepper_coords = [np.random.randint(0, i - 1, num_pepper) for i in image.shape]
    noisy_image[pepper_coords[0], pepper_coords[1]] = 0  # Pepper noise is black (0)

    return noisy_image


image = cv2.imread('images/native-bee.png')

# Check if image was loaded successfully
if image is None:
    print(f"Error: Unable to load image 'path_to_your_image.jpg'")
else:
    # Add salt-and-pepper noise
    noisy_image = add_salt_and_pepper_noise(image, salt_prob=0.02, pepper_prob=0.02)

    # Display the result
    cv2.imshow('Noisy Image', noisy_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
